In [9]:
from bs4 import BeautifulSoup
import requests

# Url of website
url='https://www.immoweb.be/en/classified/house/for-sale/seraing/4100/8771411'
# I send my HTTP request with a "GET" to the site server to identify in the url
r = requests.get(url)
# I display the requested url and the return of the server
print(url, r.status_code)
# I ask beautifulSoup to keep in a soup variable the web page to scrape (url) an html script
soup = BeautifulSoup(r.content,'lxml')
soup

property_dict = {}

for elem in soup.find_all('tr'):
    if elem.th:
        if elem.th.string:
            print(elem.th.string.strip())
            if elem.td.span:
                print(elem.td.span.string)
            else:
                print(elem.td.string)

https://www.immoweb.be/en/classified/house/for-sale/seraing/4100/8771411 200
Available as of

                                    Upon exchange of deeds
                                
Building condition

                                    Good
                                
Facades

                                    3
                                
Living area
None
Kitchen type

                                    Semi equipped
                                
Bedrooms
2
Bedroom 1 surface
None
Bedroom 2 surface
None
Bathrooms
1
Toilets
1
Basement
Yes
                                
Surface of the plot
None
Connection to sewer network

                                        Connected
                                    
Gas, water & electricity

                                        Yes
                                    
Terrace
Yes
                                
Primary energy consumption
None
Energy class

                                                              